In [ ]:
# Step 1: Imports
!pip install gradio SpeechRecognition gtts pydub --quiet

import json
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder
import gradio as gr
import os
from google.colab import files
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment

# Step 2: Upload Multiple JSON Files
uploaded = files.upload()

intents = []
for filename in uploaded:
    with open(filename, 'r') as f:
        data = json.load(f)
        if isinstance(data, list):
            for entry in data:
                if all(k in entry for k in ("intent", "examples", "responses")):
                    intents.append(entry)
                else:
                    print(f"⚠️ Skipped invalid entry in {filename}: {entry}")
        else:
            print(f"⚠️ Skipped file (not a list): {filename}")

# Step 3: Prepare Data
sentences, labels = [], []
responses = {}
for intent in intents:
    for example in intent["examples"]:
        sentences.append(example)
        labels.append(intent["intent"])
    responses[intent["intent"]] = intent["responses"]

lbl_encoder = LabelEncoder()
lbl_encoder.fit(labels)
labels_encoded = lbl_encoder.transform(labels)

# Step 4: Tokenization & Padding
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

# Step 5: LSTM Model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(len(set(labels)), activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(padded_sequences, np.array(labels_encoded), epochs=200, verbose=0)

# Step 6: Chat Functions
def predict_intent(message):
    seq = tokenizer.texts_to_sequences([message])
    padded = pad_sequences(seq, maxlen=max_len, truncating='post')
    pred = model.predict(padded, verbose=0)
    intent = lbl_encoder.inverse_transform([np.argmax(pred)])[0]
    return intent

def chatbot_response(message):
    intent = predict_intent(message)
    return random.choice(responses[intent])

# Step 7: Voice Chat Handler
def transcribe_and_chat(audio, text_input, history=[]):
    text = text_input

    if audio is not None:
        recognizer = sr.Recognizer()
        audio_file_path = audio
        with sr.AudioFile(audio_file_path) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data)
            except:
                return history + [("Voice Input", "Sorry, I couldn't understand that.")], None

    if not text:
        return history, None

    response = chatbot_response(text)
    history.append((text, response))

    # Convert response to speech using gTTS
    tts = gTTS(response)
    tts.save("response.mp3")
    sound = AudioSegment.from_mp3("response.mp3")
    sound.export("response.wav", format="wav")

    return history, "response.wav"

# Step 8: Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 LSTM Chatbot with Voice Support")

    chatbot_ui = gr.Chatbot(label="Chat")
    state = gr.State([])

    with gr.Row():
        text_in = gr.Textbox(label="Type a message or record below")
        audio_in = gr.Audio(label="🎙️ Voice Input", type="filepath")

    audio_out = gr.Audio(label="🔊 Bot Response")

    send_btn = gr.Button("Send")

    send_btn.click(
        fn=transcribe_and_chat,
        inputs=[audio_in, text_in, state],
        outputs=[chatbot_ui, audio_out]
    )

demo.launch(share=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 2.3 MB/s eta 0:00:00


Saving Batcht 6 Faculty Names and Contact Numbers.json to Batcht 6 Faculty Names and Contact Numbers.json
Saving test.json to test.json


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
<ipython-input-1-1833319196>:114: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(label="Chat")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c4d8468af26687577a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
